In [1]:
from task_cls import (
    PoliceCallBasicInfo,
    SafetySuggestionProvided
    
)

from voice_agent_flow.node import AgentNode
from voice_agent_flow.llms import create_pydantic_azure_openai
from voice_agent_flow.llms import create_ollama_model
import nest_asyncio
nest_asyncio.apply()

from voice_agent_flow.runner import AgentRunner

model = create_pydantic_azure_openai('gpt-4o-mini')

instruction = """
You are a police call center agent(working at 110). You task is to talk with caller via telephone to collection information.
You resopnse should be berief and direct to the point.
"""

agents = {
    "police_call_basic_info": AgentNode(
        name="police_call_basic_info",
        model=model,
        instruction=instruction,
        task_cls= PoliceCallBasicInfo,
        step_instruction="Collect basic information about the police call including case location, case type, description and caller name. ask the question one at a time, do not ask multiple questions in one message.",
        examples=["请问您遇到什么紧急情况？ / 发生在哪里？/ 能简单描述一下吗？/ 您的姓名是？"],
        ),
    "safety_suggestion": AgentNode(
        name="safety_suggestion",
        model=model,
        instruction=instruction,
        task_cls= SafetySuggestionProvided,
        step_instruction="Based on the collected information, provide safety suggestion(A clear command to keep safe) to the caller. After the caller responded to your suggestion, create the schema.",
        examples=["请您保持冷静，.......(provide clear safety suggestion)", "请您打开窗户，保持空气流通"],
        ),
    }

runner = AgentRunner(
    agents=agents, 
    entry_agent_name="police_call_basic_info",
    ending_message="好的，我们这就派人过去处理您的情况，请您保持电话畅通！"
)  
       
     
     

In [2]:
runner.run("喂，你要，我要报警")

'请问您遇到什么紧急情况？'

In [3]:
runner.run("有人在外面撬我家门锁")

'好的，请问这个事情发生在哪里？'

In [4]:
runner.run("背景海淀区天地大厦")

'明白了。请您能简单描述一下发生的具体情况吗？'

In [5]:
runner.run("就是这样，一直在敲，哐哐哐")

'好的，感谢您的描述。请问您方便告诉我您的姓名吗？'

In [6]:
runner.run("我叫王小明")

Collected information: {'case_location': '海淀区天地大厦', 'case_type': '门锁撬开', 'description': '有人在外面撬我家门锁，一直在敲，哐哐哐。', 'caller_name': '王小明'}
Transfer to safety_suggestion


'好的，王小明。请您保持冷静，尽量不要去开门或与外面的人接触。寻找一个安全的地方，比如房间里锁上门，等待警方到来。'

In [7]:
runner.run("是的")

Collected information: {'suggestion_provided': True}
All tasks completed.


'好的，我们这就派人过去处理您的情况，请您保持电话畅通！'

In [8]:
runner.show_information()

[{'case_location': '海淀区天地大厦',
  'case_type': '门锁撬开',
  'description': '有人在外面撬我家门锁，一直在敲，哐哐哐。',
  'caller_name': '王小明'},
 {'suggestion_provided': True}]